# Setup

In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"
import pandas as pd
try:
    # %tensorflow_version only exists in Colab.
    %tensorflow_version 2.x
except Exception:
    pass

# TensorFlow ≥2.0 is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

%load_ext tensorboard

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)
tf.random.set_seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

# Where to save the figures
PROJECT_ROOT_DIR = "."
CHAPTER_ID = "deep"
IMAGES_PATH = os.path.join(PROJECT_ROOT_DIR, "images", CHAPTER_ID)
os.makedirs(IMAGES_PATH, exist_ok=True)

def save_fig(fig_id, tight_layout=True, fig_extension="png", resolution=300):
    path = os.path.join(IMAGES_PATH, fig_id + "." + fig_extension)
    print("Saving figure", fig_id)
    if tight_layout:
        plt.tight_layout()
    plt.savefig(path, format=fig_extension, dpi=resolution)

# Input

In [9]:
# loading the Fashion-MNIST dataset
(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()

# trimming the data since it takes lot of time
X_train_full = X_train_full[:30000]
y_train_full = y_train_full[:30000]

X_test = X_test[:5000]
y_test = y_test[:5000]

# scaling the dataset
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0

# dividing the dataset into traingin and validation set
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]


# scaling the dataset
pixel_means = X_train.mean(axis=0, keepdims=True)
pixel_stds = X_train.std(axis=0, keepdims=True)
X_train_scaled = (X_train - pixel_means) / pixel_stds
X_valid_scaled = (X_valid - pixel_means) / pixel_stds
X_test_scaled = (X_test - pixel_means) / pixel_stds

# Regularisers

## L1 and L2 Regularisers

In [10]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
# or l1(0.1) for ℓ1 regularization with a factor or 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

In [11]:
# defining the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),                    
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax")
                      
])

# compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 20

# training the model
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 190us/sample - loss: 0.5078 - accuracy: 0.8216 - val_loss: 0.4589 - val_accuracy: 0.8372
Epoch 2/20
25000/25000 [==============================] - 4s 157us/sample - loss: 0.3466 - accuracy: 0.8708 - val_loss: 0.3888 - val_accuracy: 0.8622
Epoch 3/20
25000/25000 [==============================] - 4s 157us/sample - loss: 0.2890 - accuracy: 0.8927 - val_loss: 0.3846 - val_accuracy: 0.8672
Epoch 4/20
25000/25000 [==============================] - 4s 152us/sample - loss: 0.2574 - accuracy: 0.9034 - val_loss: 0.3939 - val_accuracy: 0.8688
Epoch 5/20
25000/25000 [==============================] - 4s 149us/sample - loss: 0.2290 - accuracy: 0.9124 - val_loss: 0.4084 - val_accuracy: 0.8686
Epoch 6/20
25000/25000 [==============================] - 4s 150us/sample - loss: 0.2111 - accuracy: 0.9208 - val_loss: 0.4452 - val_accuracy: 0.8684
Epoch 7/20
25000/25000 [===========================

In [12]:
# defining the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])

# compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 20

# training the model
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 193us/sample - loss: 2.5399 - accuracy: 0.8052 - val_loss: 0.9126 - val_accuracy: 0.7862
Epoch 2/20
25000/25000 [==============================] - 4s 152us/sample - loss: 0.7752 - accuracy: 0.8159 - val_loss: 0.7593 - val_accuracy: 0.8116
Epoch 3/20
25000/25000 [==============================] - 4s 155us/sample - loss: 0.7344 - accuracy: 0.8228 - val_loss: 0.7164 - val_accuracy: 0.8322
Epoch 4/20
25000/25000 [==============================] - 4s 155us/sample - loss: 0.7153 - accuracy: 0.8274 - val_loss: 0.7034 - val_accuracy: 0.8458
Epoch 5/20
25000/25000 [==============================] - 4s 146us/sample - loss: 0.7050 - accuracy: 0.8330 - val_loss: 0.6928 - val_accuracy: 0.8398
Epoch 6/20
25000/25000 [==============================] - 4s 151us/sample - loss: 0.6991 - accuracy: 0.8356 - val_loss: 0.7037 - val_accuracy: 0.8392
Epoch 7/20
25000/25000 [===========================

In [14]:
# defining the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l1(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l1(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l1(0.01))
])

# compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 20

# training the model
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 6s 226us/sample - loss: 10.4535 - accuracy: 0.7128 - val_loss: 2.3670 - val_accuracy: 0.7484
Epoch 2/20
25000/25000 [==============================] - 4s 158us/sample - loss: 2.2007 - accuracy: 0.7482 - val_loss: 2.0710 - val_accuracy: 0.7684
Epoch 3/20
25000/25000 [==============================] - 4s 155us/sample - loss: 2.0189 - accuracy: 0.7740 - val_loss: 1.9575 - val_accuracy: 0.7858
Epoch 4/20
25000/25000 [==============================] - 4s 161us/sample - loss: 1.9162 - accuracy: 0.7841 - val_loss: 1.8761 - val_accuracy: 0.7898
Epoch 5/20
25000/25000 [==============================] - 4s 160us/sample - loss: 1.8307 - accuracy: 0.7861 - val_loss: 1.7852 - val_accuracy: 0.8024
Epoch 6/20
25000/25000 [==============================] - 4s 156us/sample - loss: 1.7537 - accuracy: 0.7919 - val_loss: 1.7070 - val_accuracy: 0.7960
Epoch 7/20
25000/25000 [==========================

In [15]:
# defining the model
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l1_l2(0.005, 0.005)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l1_l2(0.005, 0.005)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l1_l2(0.005, 0.005))
])

# compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 20

# training the model
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 6s 243us/sample - loss: 7.2736 - accuracy: 0.7564 - val_loss: 1.7501 - val_accuracy: 0.7890
Epoch 2/20
25000/25000 [==============================] - 4s 166us/sample - loss: 1.6508 - accuracy: 0.7890 - val_loss: 1.5710 - val_accuracy: 0.8048
Epoch 3/20
25000/25000 [==============================] - 4s 162us/sample - loss: 1.5426 - accuracy: 0.7969 - val_loss: 1.4889 - val_accuracy: 0.8100
Epoch 4/20
25000/25000 [==============================] - 4s 161us/sample - loss: 1.4722 - accuracy: 0.8005 - val_loss: 1.4377 - val_accuracy: 0.8162
Epoch 5/20
25000/25000 [==============================] - 4s 174us/sample - loss: 1.4212 - accuracy: 0.8052 - val_loss: 1.3921 - val_accuracy: 0.8158
Epoch 6/20
25000/25000 [==============================] - 4s 177us/sample - loss: 1.3840 - accuracy: 0.8073 - val_loss: 1.3574 - val_accuracy: 0.8150
Epoch 7/20
25000/25000 [===========================

## Dropout

In [18]:
# defining a model applying dropout regularization
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])

# compiling the model
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 20

# training the model
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

Train on 25000 samples, validate on 5000 samples
Epoch 1/20
25000/25000 [==============================] - 5s 199us/sample - loss: 0.6865 - accuracy: 0.7756 - val_loss: 0.4478 - val_accuracy: 0.8390
Epoch 2/20
25000/25000 [==============================] - 4s 166us/sample - loss: 0.4678 - accuracy: 0.8282 - val_loss: 0.3633 - val_accuracy: 0.8664
Epoch 3/20
25000/25000 [==============================] - 4s 166us/sample - loss: 0.4142 - accuracy: 0.8457 - val_loss: 0.3669 - val_accuracy: 0.8686
Epoch 4/20
25000/25000 [==============================] - 4s 153us/sample - loss: 0.3901 - accuracy: 0.8542 - val_loss: 0.3524 - val_accuracy: 0.8732
Epoch 5/20
25000/25000 [==============================] - 4s 157us/sample - loss: 0.3704 - accuracy: 0.8570 - val_loss: 0.3779 - val_accuracy: 0.8692
Epoch 6/20
25000/25000 [==============================] - 4s 156us/sample - loss: 0.3613 - accuracy: 0.8652 - val_loss: 0.3447 - val_accuracy: 0.8782
Epoch 7/20
25000/25000 [===========================